In [170]:
import math

class Node:
    def __init__(self, node, x, y, node_name):
        self.id = node
        self.x = x
        self.y = y
        self.node_name = node_name
        self.adj = {}
    
    def calculate_dist_to(self, neighbour):
        return (neighbour.get_x() - self.x, neighbour.get_y() - self.y)
    
    def add_neighbour(self, neighbour):
        self.adj[neighbour] = self.calculate_dist_to(neighbour)
    
    def get_id(self):
        return self.id
    
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def get_degree(self):
        return len(self.adj)
    
    def get_neighbours(self):
        return self.adj
    
    def __repr__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adj]) + '\n'
    
    def __str__(self):
        return self.__repr__()
    

In [171]:
# This class implements an undirected graph
class Graph:
    def __init__(self):
        self.node_map = {}
        self.num_nodes = 0
    
    def add_node(self, node_id, x, y, node_name):
        self.num_nodes = self.num_nodes + 1
        new_node = Node(node_id, x, y, node_name)
        self.node_map[node_id] = new_node
        return new_node
    
    def get_nodes(self):
        return self.node_map.values()
    
    def is_valid_node_id(self, node_id):
        if (node_id not in self.node_map):
            return False
        return True
    
    def add_edge(self, node_one, node_two):
        if (self.is_valid_node_id(node_one) and self.is_valid_node_id(node_two)):
            self.node_map[node_one].add_neighbour(self.node_map[node_two])
            self.node_map[node_two].add_neighbour(self.node_map[node_one])
    
    def get_edges(self, node):
        return self.node_map[node]
    
    def get_degree(self, node):
        return self.node_map[node].get_degree()
    
    def __repr__(self):
        result = ''
        for node in self.node_map.values():
            result += str(node)
        return result

In [172]:
class Dijkstra:
    
    def __init__(self, graph, source):
        self.dist_to = [] # distance of shortest s->v path
        self.edge_to = [] # last edge on shortest s->v path
        self.pq = [] # priority queue of vertices

In [173]:
import requests
import json

r = requests.get("http://showmyway.comp.nus.edu.sg/getMapInfo.php?Building=COM1&Level=1")

body = r.json()

g = Graph()

for node in body["map"]:
    node_id = int(node['nodeId'])
    x = int(node['x'])
    y = int(node['y'])
    node_name = node['nodeName']
    link_to = node['linkTo'].split(', ')
    
    g.add_node(node_id, x, y, node_name)
    
    for i in range(len(link_to)):
        g.add_edge(node_id, int(link_to[i]))

print(g)

1 adjacent: [2]
2 adjacent: [3, 1]
3 adjacent: [2, 4]
4 adjacent: [3, 5]
5 adjacent: [6, 4]
6 adjacent: [8, 7, 5]
7 adjacent: [6]
8 adjacent: [6]

